In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Script to prepare credit card data for Superset by pre-computing
aggregations and uploading them to HDFS/Hive
"""

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

print("Starting data preparation for Superset...")

# Create Spark session
spark = SparkSession.builder \
    .appName("Prepare Credit Card Data for Superset") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

print(f"Spark version: {spark.version}")
print(f"Spark master: {spark.sparkContext.master}")

# Define schema
schema = StructType([
    StructField("Tanggal", StringType(), True),
    StructField("Provinsi", StringType(), True),
    StructField("Nilai_Server", DoubleType(), True),
    StructField("Nilai_Kartu", DoubleType(), True),
    StructField("Nilai_Registered", DoubleType(), True),
    StructField("Nilai_Unregistered", DoubleType(), True)
])

# Try reading from different locations
file_paths = [
    "/home/jovyan/data/DataFixABD.csv",
    "/home/jovyan/work/DataFixABD.csv",
    "/data/DataFixABD.csv",
    "hdfs:///data/credit-card-data/DataFixABD.csv"
]

df = None
for path in file_paths:
    try:
        print(f"Trying to read from {path}")
        df = spark.read.csv(path, header=True, schema=schema)
        count = df.count()
        print(f"Successfully loaded {count} rows from {path}")
        break
    except Exception as e:
        print(f"Error: {str(e)}")

if df is None:
    raise Exception("Could not load data from any location")

# Process the data
print("Processing the data...")
df = df.withColumn("tanggal", to_date(col("Tanggal")))
df = df.withColumn("tahun", year(col("tanggal")))
df = df.withColumn("bulan", month(col("tanggal")))
df = df.withColumn("nama_bulan", date_format(col("tanggal"), "MMMM"))
df = df.withColumn("hari", dayofmonth(col("tanggal")))
df = df.withColumn("hari_dalam_minggu", date_format(col("tanggal"), "EEEE"))

# Create a Hive table with the processed data
print("Creating Hive table...")
df.write.format("hive").mode("overwrite").saveAsTable("default.credit_card_data")

# Verify the table was created
print("Verifying Hive table...")
spark.sql("SHOW TABLES").show()
row_count = spark.sql("SELECT COUNT(*) FROM credit_card_data").collect()[0][0]
print(f"Created table with {row_count} rows")

# Also save as parquet for better performance
print("Saving data as Parquet...")
df.write.format("parquet").mode("overwrite").save("hdfs:///data/credit-card-data/processed")

print("Data preparation complete!")

# Close the Spark session
spark.stop()